In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from census import Census
from pprint import pprint
# import gmaps

# Google API Key
from config import gkey

# Incorporated citipy to determine latitude and longitude based on city 
from citipy import citipy


In [2]:
# Import CSV and create a df for cities
us_cities_to_load="us_cities.csv"

us_cities_df=pd.read_csv(us_cities_to_load)

us_cities_df.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440


In [3]:
us_cities_df=us_cities_df.rename(columns={'city':'City','state_name':'State','lat':'Latitude','lng':'Longitude',
                                          'population':'Population'})

us_cities_df.head()

,City,city_ascii,state_id,State,county_fips,county_name,Latitude,Longitude,Population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440


In [4]:
us_cities_df.drop(us_cities_df.columns[[1,2,4,5,9,10,11,12,13,14,15,16]], axis=1, inplace=True)

us_cities_df.head()

,City,State,Latitude,Longitude,Population
0,New York,New York,40.6943,-73.9249,18713220
1,Los Angeles,California,34.1139,-118.4068,12750807
2,Chicago,Illinois,41.8373,-87.6862,8604203
3,Miami,Florida,25.7839,-80.2102,6445545
4,Dallas,Texas,32.7936,-96.7662,5743938


In [5]:
len(us_cities_df.index)


28338

In [6]:
us_cities_df.shape


(28338, 5)

In [7]:
us_cities_df=us_cities_df.loc[us_cities_df['Population']>=1000000]
us_cities_df.shape

(50, 5)

In [8]:
us_cities_df.head()

,City,State,Latitude,Longitude,Population
0,New York,New York,40.6943,-73.9249,18713220
1,Los Angeles,California,34.1139,-118.4068,12750807
2,Chicago,Illinois,41.8373,-87.6862,8604203
3,Miami,Florida,25.7839,-80.2102,6445545
4,Dallas,Texas,32.7936,-96.7662,5743938


In [9]:
us_cities_df["Restaurant Name"]=""
us_cities_df["Restaurant Type"]=""
us_cities_df["Rating"]=""
us_cities_df["Number of Reviews"]=""

us_cities_df.head()

,City,State,Latitude,Longitude,Population,Restaurant Name,Restaurant Type,Rating,Number of Reviews
0,New York,New York,40.6943,-73.9249,18713220,,,,
1,Los Angeles,California,34.1139,-118.4068,12750807,,,,
2,Chicago,Illinois,41.8373,-87.6862,8604203,,,,
3,Miami,Florida,25.7839,-80.2102,6445545,,,,
4,Dallas,Texas,32.7936,-96.7662,5743938,,,,


In [17]:
us_cities_df=us_cities_df.sample(10)

us_cities_df

,City,State,Latitude,Longitude,Population,Restaurant Name,Restaurant Type,Rating,Number of Reviews
17,Denver,Colorado,39.7621,-104.8759,2876625,,,,
2,Chicago,Illinois,41.8373,-87.6862,8604203,,,,
48,New Orleans,Louisiana,30.0687,-89.9288,1020886,,,,
42,Jacksonville,Florida,30.3322,-81.6749,1181496,,,,
46,Memphis,Tennessee,35.1046,-89.9773,1066967,,,,
47,Raleigh,North Carolina,35.8325,-78.6435,1038738,,,,
21,Baltimore,Maryland,39.3051,-76.6144,2106068,,,,
25,St. Louis,Missouri,38.6358,-90.2451,2024074,,,,
8,Washington,District of Columbia,38.9047,-77.0163,5379184,,,,
41,Providence,Rhode Island,41.8230,-71.4187,1203230,,,,


In [ ]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Define Parameters
target_coordinates = "40.6943, -73.9249"
# target_coordinates = "43.6187102, -116.2146068"
target_search = "mexican"
# meters is the units
target_radius = 1000
target_type = "restaurant"

In [11]:
# geocoordinates


target_search = "mexican"
# meters is the units
target_radius = 1000
target_type = "restaurant"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# frame_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=input=location=33.753746%2C+-84.386330&keyword=mexican&radius=1000&type=restaurant&key=API_KEY_HERE"  

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [12]:
# print the response url, avoid doing for public github repos in order to avoid exposing key

#print(response.url)

In [13]:
# convert response to json
places_data = response.json()
    
# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))


{
    "html_attributions": [],
    "next_page_token": "Aap_uEC5adiMRoR8RnCvmQjVixITb05o9AEKfeBOv8ZcHJDY0zJDSeVmI92byDUMHMZqiiuFooZw5DwtgPP6ugRuweigMVD64vHzvUF0WVbgHW6k7DLYvWI622p17PvIh5zSIvuR5ITs15tiMUK-aYlgiyP9DRPnmRTrThokD_Tf4pjKKGhrpcNv0jcD237rrNB-qtEp1NfTeRevmrAGdKFbVXMNQwaYi9tByinM4BaM_DlYXX2KF3jOQd_JNzEAsn1qH5e85peQEtTUInUxBi2Isdn44BfbpVXwgOdMIY3LwvjcaCrGBTqynj2iwvyxhOiMw4UdolJjtQDbuGvTuwt6uaSj8XdZpxBRFFNH3LTkgoE0Iotx8e4ngQdPATvMxKu2TQpDdm9O1tFubcmzDf17UDJs-Mf0eoD6ydvsWspfPkkN1NObuL96osIJCWFf9SjBrJT2g-0P",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 40.7031423,
                    "lng": -73.9218599
                },
                "viewport": {
                    "northeast": {
                        "lat": 40.70453567989272,
                        "lng": -73.92047047010728
                    },
                    "southwest": {
                        "lat": 40

In [ ]:
places_df=pd.read_json('https://maps.googleapis.com/maps/api/place/nearbysearch/json')

places_df.head()


In [ ]:
with open('https://maps.googleapis.com/maps/api/place/nearbysearch/json','r') as f:
    data = json.loads(f.read())

In [ ]:
# convert response to json
places_data = response.json()
    
# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))
# places_data_json = pd.read_json('base_url')

In [ ]:
import requests

url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Museum%20of%20Contemporary%20Art%20Australia&inputtype=textquery&fields=formatted_address%2Cname%2Crating%2Copening_hours%2Cgeometry&key=API_KEY_HERE"

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)

In [ ]:
# Print the name and address of the first restaurant that appears
# print(places_data["results"][0]["name"]),(places_data["results"][0]["vicinity"])
# print(places_data["results"][0]["vicinity"])
# print(places_data["results"][10]["name"])
# print(places_data["results"][10]["vicinity"])


# name = []
# rating = []
# user_ratings_total = []
# latitude = []
# longatiude = []
# business_satus = [] 
# place_id = []








In [ ]:
# df_nested_list = pd.json_normalize(base_url, record_path =['name'])


# df = pd.read_json(base_url)
# df.head()


# build our data frame columns
name = []
rating = []
user_ratings_total = []
lat = []
lng = []
business_status = []
price_level = []

# Loop through the list of cities and perform a request for data on each
# for i, row in places_data.iterrows():
# # for restaurnts in places_data:
#     response = requests.get(base_url, params=params).json()
    
    
    
name.append(response["results"][i]["name"])
#     rating.append(response["results"][i]["rating"])
#     user_ratings_total.append(response["results"][i]["user_ratings_total"])
#     lat.append(response["results"][i]["geometry"]["location"]["lat"])
#     lng.append(response["results"][i]["geometry"]["location"]["lng"])
#     business_status.append(response["results"][i]["business_status"])
#     price_level.append(response["results"][i]["price_level"])

#     lat.append(response['coord']['lat'])
#     temp.append(response['main']['temp'])
print(name)

In [ ]:
# atl_df=pd.DataFrame({"places_data":places_data})
# atl_df["place_id"] = ""
# atl_df["name"] = ""
# atl_df["rating"] = ""
# atl_df["user_ratings_total"] = ""
# atl_df["lat"] = ""
# atl_df["lng"] = ""
# atl_df["business_status"] = ""
# create a data frame from cities, lat, and temp
# weather_dict = {
#     "city": cities,
#     "lat": lat,
#     "temp": temp
# }
# weather_data = pd.DataFrame(weather_dict)
# weather_data.head()



# atl_dict = {
# #     "Place ID":place_id,
#     "Name":name,
#     "Rating":rating,
#     "Date":dates,
#     "User Total Rating":user_ratings_total,
#     "Latitude":lat,
#     "Longatiude":lng,
#     "Business Status":business_status,
#     "Price Level":price_level,


places_data_df=pd.DataFrame(places_data)
places_data_df.head()


In [ ]:
url="https://api.yelp.com/v3/businesses/search?term=delis&latitude=37.786882&longitude=-122.399972"
# Make request and store response
response = requests.get(url)

# Verify status code
response.status_code